# MIE562 Project - Truck Scheduling - MIP Model

**Team 5 - Dylan Camus, Ryan Do, Fan Jia, Matheus Magalhaes, Sugumar Prabhakaran**
**Date: 8 Nov 2020

In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import Model, GRB, quicksum
from random import randint, seed

seed(0)

## Introduction

### 1. Sets
   * Each container $k$ is in a set of containers $K$: 
       * $k \in K$, where $K = \left \{1,2,3,\ldots,k  \right \}$
       
   * Each carrier $j$ is in a set of carriers $J$: 
       * $j \in J$, where $J = \left \{1,2,3,\ldots,j  \right \}$
       
   * Each chassis $c$ is in a set of chassis, $C$: 
       * $c \in C$, where $C = \left \{1,2,3,\ldots,c  \right \}$
 
   * $L$ is the set of travel legs: $L = \left \{1,2,3\right\}$, where:
       * $1$ : terminal to transloading facility leg
       * $2$ : terminal to stack leg
       * $3$ : stack to transloading facility leg

In [2]:
#Define Sets and recieve input for initial "instances"
J = np.arange(1,int(input("Number of Carriers:"))+1)
K= []

for j in J:
    containers = int(input("Containers in Carrier #{}:".format(j)))
    for k in range(1,containers + 1):
        K.append((j,k))    
    
C = np.arange(1,int(input("Number of Chassis:"))+1)

L = np.arange(1,4)

print("\nCarriers J:", J)
print("Containers K (j,k):", K)
print("Chassis C:", C)
print("Legs L:", L)

Number of Carriers:2
Containers in Carrier #1:10
Containers in Carrier #2:12
Number of Chassis:8

Carriers J: [1 2]
Containers K (j,k): [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12)]
Chassis C: [1 2 3 4 5 6 7 8]
Legs L: [1 2 3]


In [3]:
#create a list of tuples containing every iteration of k,l,c
A = [(k,l,c) for k in K for l in L for c in C]

### 2. Parameters

   * $\Phi_{kl}$  : Time duration of leg l for container k [days] 
   * $D_{ll'}$: Delay required for a chassis to start leg l' after completing l (including travel) [days]
   * $M$     : Some large number
   * $R_k$   : Release date for container $k$ [days]
   
   * $T_j^{''}$: free period before demurrage cost for carrier j at terminal [days]
   
   * $T_j$: demurrage cost/unit time for carrier j at terminal [($/days)/container]
   
   * $S$     : fixed cost at stack [$/container]
   
   * ${S}'$  : variable cost at stack [($/day)/container]
   
   * $\delta_j^{''}$: free period before detention cost for containers of carrier j [days]
   
   * $\delta_j$: detention cost/unit time for containers of carrier j [($/days)/container]
   
   * $Z_{jk}$: 1 if container $k$ belongs to carrier $j$
   * $\rho_k$ : priority factor of container k (higher value is higher priority)
   
   

In [40]:
#Set Parameters

#Time duration of leg l for container k [days], k in K and l in L
Phi_kl = {k:randint(0,9)/10 for k in K} #...NOT SURE HOW TO REPRESENT THIS???
D_llprime = 0                         #...NOT SURE I UNDERSTAND THIS ONE

M = 1000000                        # some large number
R = {k:randint(0,5) for k in K}  # release date for container k[days]

# free period before demurrage cost for containers from carrier j at terminal [days]
T_prime = {j: randint(3,7) for j in J}
# demurrage cost per container of carrier j after free period [($/day)/container]
T = {j: randint(1,9)*50 for j in J} #CAN WE ASSUME FIXED? for all carriers?

S = 200                            # $200 fixed cost at stack
S_prime = 20                       # $20/day/container variable cost at stack

# free period before detention cost for containers from carrier j [days]
Delta_prime = {j: randint(3,7) for j in J} 
# detention cost per container of carrier j [($/day)/container]
Delta = {j: randint(1,3)*100 for j in J}  # CAN WE ASSUME FIXED? for all carriers?

Z = {(j,k):1 for j, k in K}     # DO WE NEED THIS? if we use tuples of (j,k)?
rho = {k:randint(1,3) for k in K}  # priority factor for container k

In [5]:
#Create a new model
model = gp.Model("Truck_Scheduling")

Using license file /home/sugumarprabhakaran/gurobi.lic
Academic license - for non-commercial use only


### 3. Decision Variables

   * Binary Variable: $x_{klc}$
   $\begin{equation}
  =\left\{
  \begin{array}{@{}ll@{}}
    1, & \text{if}\ \text{container k travels leg l on chassis c} \\
    0, & \text{otherwise}
  \end{array}\right.
\end{equation} $
   * Start time $s$ of container $k$ on leg $l$ on chassis $c$: $s_{klc}$, where  $s_{klc} \geq 0$

In [6]:
#Create decision variables
x = model.addVars(A, vtype=GRB.BINARY, name="x_klc")
s = model.addVars(A, vtype=GRB.INTEGER, name="s_klc")
c_dem = model.addVar(vtype=GRB.INTEGER, name="demurrage cost")
c_det = model.addVar(vtype=GRB.INTEGER, name="detention cost")
c_stk = model.addVar(vtype=GRB.INTEGER, name="stack cost")
c_pri = model.addVar(vtype=GRB.INTEGER, name="priority cost");

### 4. Objective Function

We want to minimize the Total Cost ($c$) = (Demurrage cost}) + (Detention cost) + (Stack cost) + (Priority penalty cost)
   
   * **objective function:** $min$ $(c)$
   
   * $c = c_{dem} + c_{det} + c_{stk} + c_{pri}$

**(1) Demurrage Cost:**

The Demurrage Cost ($c_{dem}$) refers to cost associated with keeping containers in the terminal beyond the free period allotedin days ($T_{j}^{''}$) that is different for each carrier.

If the start time for a container ($s_{klc}$) is higher than the release date ($R_{k}$) + the free period ($T_{j}^{''}$), there will be a variable cost per extra day ($T_{j}$) per container.
   
   * $c_{dem} = \sum_{k \in K}\sum_{c \in C}\sum_{l \in L}\sum_{j \in J}(T_j)\cdot max\left [(s_klc - R_k - T_{j}^{''})\cdot (\x_{\klc}?)Z_{jk}, 0  \right ]$
   
**Note.  Do we not need $x_{klc}$ to account for the $s_{klc}$?**
   
**(2) Detention Cost**

The terminal also charges a detention cost ($c_{det}$) for containers that are not returned to the terminal for processing before return to the carrier.  Similar to demurrage, there is a free period before which the containers must be returned to the terminal ($\delta_{j}^{''}$).

If the start time for a container ($s_{klc}$) and the total travel time of a container back to the terminal including processing time ($\phi_{kl}$) is greater than the release date ($R_{k}$) + the free period ($\delta_{j}^{''}$), there will be a variable cost per extra day ($T_{j}$) per container.

   * $c_{det} = \sum_{k \in K}\sum_{c \in C}\sum_{l \in {1,3}}\sum_{j \in J}(\delta_j)\cdot max\left [(s_{klc} - R_k - \phi_{kl} - \delta_{j}^{''})\cdot x_{klc}Z_{jk}, 0  \right ]$
   
**Note2.  Could it be easier if we just used a $P_{k}$ for processing time of a container?  Then $\phi$ could depend just on the legs? Might make it easier for creating the data structure for P?**

**(3) Stack Storage Cost**

The stack cost ($c_{stk}$) is the sum of the fixed cost percontainer ($S$) and the variable cost per day per container ($S^{'}$) x the number of days that a container is in the stack, which is the difference between leg 3 and leg 2 start times ($s_{klc}$).

   * $c_{stk} = \sum_{k \in K}\sum_{c \in C}x_{k2c}\cdot (S + S' (s_{k3c} - s_{k2c}))$

**(4) Priority Penalty Cost**

The priority penalty cost ($c_{pri}$) is applied for every day that a container sits in the terminal beyond the release day ($R_{k}$).  Since the priority factor per container ($\rho_{k}$) is higher for higher priority containers, the associated penalty costs will be higher.

   * $c_{det} = \sum_{k \in K}\sum_{c \in C}\sum_{l \in \{{1,2}\}}\sum_{j \in J}\rho_k\cdot (s_{klc} - R_k)\cdot x_{klc}Z_{jk}$
   
**Note 3. Should we not include time spent at the stack as well towards this cost since priority containers sitting at the stack would still be suboptimal?**

In [39]:
#Define objective function
model.modelSense = GRB.MINIMIZE
model.setObjective(c_dem + c_det + c_stk + c_pri)

#c_dem = quicksum(
#    T[j]*max((s[k,l,c] - R[k] - T_prime[j])*x[k,l,c]*Z[k],0) 
#    for k in K for c in C for l in L for j in J if l !=3)
#Note1.  Having trouble with the max function in the quicksum...



c_stk = quicksum(x[k,2,c]*(S + S_prime*(s[k,3,c] - s[k,2,c]))
                 for k in K for c in C)

c_pri = quicksum(rho[k]*(s[k,l,c] - R[k])*x[k,l,c]*Z[k]
                for k in K for c in C for l in L if l !=3)
#Note3. I think we need an element for delays at stack for c_pri

### 5. Constraints

**(2) Release date constraint**

Start time ($s_{klc}$) must be after release date ($R_k$) for every container that leaves terminal on either leg 1 or leg 2 (ie. $x_{klc}=1$):

   * $s_{klc} \geq R_{k}x_{klc}$, &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\forall k \in K$, $l\in L$, $c \in C$ 

In [31]:
model.addConstrs(s[k,l,c]>= R[k]*x[k,l,c] for k in K for l in L for c in C);

**(3) Containers going through leg 2 must go through leg 3**

Each container k will have a x_klc value of 1 or 0 for both leg 2 and leg 3 depending on if they travel through them or not:

   * $\sum_{c \in C}(x_{k2c}) = \sum_{c \in C}(x_{k3c})$,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\forall k \in K$

In [9]:
model.addConstrs(quicksum(x[k,2,c] for c in C)==quicksum(x[k,3,c] for c in C) for k in K);

**(4) Container must go through leg 1 OR leg 2 AND 3**

Each container ($k$) will have $x_klc = 1$ for either leg 1 or leg 2 so the sum of the two must be 1 for all containers ($k$) on all chassis ($c$).

   * $\sum_{c \in C}(x_{k1c}) + \sum_{c \in C}(x_{k2c}) = 1$,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\forall k \in K$

In [10]:
model.addConstrs(
    (quicksum(x[k,1,c] for c in C) + quicksum(x[k,2,c] for c in C))==1 for k in K
);

**(5) Precedence Constraint: Leg 2 must be before Leg 3**

Each container k will have a x_klc value of 1 for either leg 1 or leg 2 so the sum of the two must be 1 for all containers (k) on all chassis (c).

   * $M\sum_{c \in C}(x_{k1c}) + \sum_{c \in C}(x_{k3c}s_{k3c}) \geq = \sum_{c \in C}(x_{k2c}s_{k2c}) + \Phi_{k2}$,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\forall k \in K$
   

In [11]:
model.addConstrs(
    (quicksum(x[k,1,c] for c in C) + quicksum(x[k,3,c]*s[k,3,c] for c in C)) >= 
    (quicksum(x[k,2,c]*s[k,2,c] for c in C) + S) for k in K
);
#Note... Need to replace 'S' with 'Phi_k2' but haven't created that parameter yet

**(6) Domain of $s_{klc}$**

The start time ($s_{klc}$) must be greater than 0:

   * $s_{klc} \geq 0$,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\forall k \in K$, $l \in L$, $c \in C$

In [12]:
model.addConstrs(s[k,l,c] >=0 for k in K for l in L for c in C);